# Breakfast Restaurant Locations
The aim of this notebook is to determine which communities in Calgary are best suited for a newly opening breakfast restaurant.

In [67]:
# Import libs
import pandas as pd
import numpy as np
import plotly.offline as py # TODO switch to online mode to generate iframe-ready graphs on plot.ly
import plotly.graph_objs as go
import folium
import json, requests

Let's start by gathering the 2016 census data, which is already grouped into communities. Additionally, since all Open Calgary data is available via the SODA API, we can select columns in our request (a bit like SQL). 

In [14]:
request_fields = ['CLASS', 'CLASS_CODE', 'COMM_CODE', 'NAME', 'COMM_STRUCTURE', 'RES_CNT', 'DWELL_CNT', 'EMPLYD_CNT', 'multipolygon']
url = 'https://data.calgary.ca/resource/mndt-3a4i.csv?$select=' + ','.join(request_fields)
census = pd.read_csv(url)
census.head()

,CLASS,CLASS_CODE,COMM_CODE,COMM_STRUCTURE,DWELL_CNT,EMPLYD_CNT,NAME,RES_CNT,multipolygon
0,Residential,1,CHA,BUILDING OUT,4177,0,CHAPARRAL,12717,MULTIPOLYGON (((-114.012060369819 50.893508382...
1,Residential,1,RED,1960s/1970s,911,0,RED CARPET,1596,MULTIPOLYGON (((-113.935199509107 51.037942708...
2,Residential,1,BDO,1960s/1970s,378,0,BONAVISTA DOWNS,919,MULTIPOLYGON (((-114.026556323391 50.948143809...
3,Residual Sub Area,4,13I,UNDEVELOPED,0,0,13I,0,MULTIPOLYGON (((-114.104975777834 50.892156937...
4,Industrial,2,ST4,EMPLOYMENT,4,0,STONEY 4,0,MULTIPOLYGON (((-113.969731915676 51.176548368...


In [ ]:
calgary_geo = 'data/Community Boundaries.geojson' # This should be identical to the multipolygon field anyways, but it saves me some hassle of transforming data for Folium's picky needs.

Let's quickly generate a map of Calgary to verify that our geojson matches our dataframe. Unfortunately, complex maps do not work natively in jupyterlab, so I have to use the workaround [here](https://github.com/python-visualization/folium/issues/812).

In [43]:
def embed_map(m):
    from IPython.display import IFrame

    m.save('index.html')
    return IFrame('index.html', width='100%', height='750px')

In [44]:
map_communities = folium.Map(location=[51.0486, -114.0708], zoom_start=11)

folium.Choropleth(
    geo_data = calgary_geo,
    data = census,
    columns = ['NAME', 'RES_CNT'],
    key_on = 'feature.properties.name',
    fill_color='YlGn'
).add_to(map_communities)
map_communities.save('comm.html')
embed_map(map_communities)

Next, let's get all the breakfast restaurants within the bounding box of each community:

In [88]:
def get_bounding_box(multipoly):
    coord_list = multipoly[16:-3].split(',')
    sw_long, sw_lat = list(map(float, coord_list[0].split(' ')))
    ne_lat, ne_long = sw_lat, sw_long
    for i in range(1,len(coord_list)):
        lat_long = filter(None, coord_list[i].split(' '))
        long_new, lat_new = list(map(float, lat_long))
        sw_lat = max(sw_lat, lat_new)
        sw_long = min(sw_long, long_new)
        ne_lat = min(ne_lat, lat_new)
        ne_long = max(ne_long, long_new)
    return (str(round(sw_lat, 8))+','+str(round(sw_long, 8)), str(round(ne_lat, 8))+','+str(round(ne_long, 8)))
    
get_bounding_box(census['multipolygon'][0])

('50.89350838,-114.05439198', '50.87569155,-114.00181285')

In [89]:
all_venues = []

for index, row in census.iterrows():
    bounding_box = get_bounding_box(row['multipolygon'])
    params = dict(
        client_id=client_id, # These have been defined in the iPython terminal at runtime to keep them out of the notebook.
        client_secret=client_secret,
        v='20180323',
        # near='Calgary, Alberta',
        limit=50,
        sw = bounding_box[0],
        ne = bounding_box[1],
        query='breakfast'
    )
    res = requests.get(url='https://api.foursquare.com/v2/venues/explore', params=params).text
    print('---')
    print(bounding_box[0])
    venues = json.loads(res)['response']['groups'][0]['items']
    print(json.loads(res))
    for venue in venues:
        venue_row = [
            row['NAME'],
            row['COMM_CODE'],
            venue['venue']['name'],
            venue['venue']['location']['lat'],
            venue['venue']['location']['lng'],
            venue['categories']
        ]
        print(venue_row)
        all_venues.append(venue_row)


venue_df = pd.DataFrame(all_venues, columns = ['Community Name', 'Community Code', 'Venue Name', 'Lat', 'Long', 'Categories'])

---
50.89350838,-114.05439198
{'response': {'groups': [{'items': [], 'name': 'recommended', 'type': 'Recommended Places'}], 'query': 'breakfast', 'headerFullLocation': 'Chaparral, Calgary', 'headerLocationGranularity': 'neighborhood', 'warning': {'text': 'There aren\'t a lot of results for "breakfast." Try something more general, reset your filters, or expand the search area.'}, 'totalResults': 0, 'headerLocation': 'Chaparral', 'suggestedBounds': {'ne': {'lat': 50.87569155, 'lng': -114.00181285}, 'sw': {'lat': 50.89350838, 'lng': -114.05439198}}}, 'meta': {'code': 200, 'requestId': '5c4a3f684c1f671d0005f96d'}}
---
51.04722243,-113.94989036
{'response': {'groups': [{'items': [], 'name': 'recommended', 'type': 'Recommended Places'}], 'query': 'breakfast', 'headerFullLocation': 'Penbrooke Meadows, Calgary', 'headerLocationGranularity': 'neighborhood', 'warning': {'text': 'There aren\'t a lot of results for "breakfast." Try something more general, reset your filters, or expand the search a

KeyboardInterrupt: 